In [1]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from scipy.signal import savgol_filter
 
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

In [2]:
#import smilestring code file
%run smilestring.ipynb
#import molecular descriptors code file
%run descriptors.ipynb
##import substatmobile and feel free to swtich between tier1_my and teir1_statmobile
#tier1 = pd.read_excel ('G# for LSR.xlsx')
#tier1_Rt=tier1.iloc[:,-1]
#tier1 = tier1.iloc[:,:-1]
#print(tier1)

#import training datadet
#import internal data
inter = pd.ExcelFile('own LC data_v2.xlsx')
inter = pd.read_excel(inter,'Comb1')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
comb = comb.reset_index(drop=True)
print(comb)

y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])
print(y)

       Project             Column       Gnum tR (min)     H    S*     A     B  \
0     GDC-0032            ACE C18  G00587744    5.659  1.00  0.02 -0.09  0.00   
1     GDC-0032            ACE C18  G01577626     6.01  1.00  0.02 -0.09  0.00   
2     GDC-0032            ACE C18  G00587678    6.138  1.00  0.02 -0.09  0.00   
3     GDC-0032            ACE C18  G02773237    6.299  1.00  0.02 -0.09  0.00   
4     GDC-0032            ACE C18  G01577625    6.451  1.00  0.02 -0.09  0.00   
...        ...                ...        ...      ...   ...   ...   ...   ...   
1785  GDC-0853  Zorbax Extend C18  G02629352    5.256  1.09  0.05  0.01 -0.04   
1786  GDC-0853  Zorbax Extend C18  G02771828    5.706  1.09  0.05  0.01 -0.04   
1787  GDC-0853  Zorbax Extend C18  G02599853    6.287  1.09  0.05  0.01 -0.04   
1788  GDC-0853  Zorbax Extend C18  G02771827    6.746  1.09  0.05  0.01 -0.04   
1789  GDC-0853  Zorbax Extend C18  G02771829    7.793  1.09  0.05  0.01 -0.04   

      C (ph=2.8)  C (ph=7.0

In [22]:
#READ DATA
X = comb.drop(['Project','Column','Gnum','tR (min)'], axis=1)
y = np.asarray(y).astype('float32')
X = np.asarray(X).astype('float32') 
#TRY LOOP
#Gnumtest = pd.DataFrame(columns = ["PredComb" + str(i) for i in range(1,len(newGnum)+1)])


#X = comb.drop(['Project','Column','Gnum','tR (min)'], axis=1)
#X['type'] = 'train'
#X_wo = X.drop(['H','S*','A','B','C (ph=2.8)','C (ph=7.0)','EB retention factor','particle size (um)','length (mm)','diameter (mm)','flow rate (mL/min)','PH'], axis=1)
#X_wo['type'] = 'train'
#test_column=pd.unique(comb['Column']).tolist()
test_pH=pd.unique(comb['PH'].tolist())
print(test_column)
print(test_pH)
y_LOO_pred=[]
y_LOO_mea=[]
LABEL=[]
# for input
test_column='Zorbax Extend C18'

temp= (comb['Column']==test_column) & (comb['PH']==4.8)

X_test = X[temp,:]
X_train= X[~temp,:]
y_train= y[~temp]
y_test= y[temp]

from sklearn.preprocessing import StandardScaler
XScaler=StandardScaler()
YScaler=StandardScaler()

# Storing the fit object for later reference (Pete learned from ChatGPT)
X_scaled=XScaler.fit_transform(X_train)
y_scaled=YScaler.fit_transform(y_train)

# Generating the standardized values of X and y (Pete learned from ChatGPT)
#X=PredictorScalerFit.transform(X)
#y_scale=TargetVarScalerFit.transform(y)
X_test_scaled=XScaler.transform(X_test)

#X_test = np.asarray(X_test.iloc[:,1:-1]).astype('float32')

# create ANN model
model = Sequential()
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# Defining the Third layer of the model
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')    
# Fitting the ANN to the Training set
model.fit(X_scaled, y_scaled ,batch_size = 50, epochs = 50, verbose=0)

#X_test = X_test.astype('float32')
# Generating Predictions on testing data
Predictions=model.predict(X_test_scaled)
Predictions=list(Predictions)
#print(len(Predictions))
#print(X_test_scaled)
#temp=PredictorScalerFit.inverse_transform(X_test_scaled)
#print(temp)


# Scaling the predicted Price data back to original price scale
Predictions=YScaler.inverse_transform(Predictions)

result = {
'test column':test_column,
'test pH':test_pH,
'RMSE':math.sqrt(mean_squared_error(y_test, Predictions))}
print(result)


y_LOO_pred = pd.DataFrame(np.asarray(Predictions).astype('float32'),columns = ['pred tR by ANN (min)'])
y_LOO_mea = pd.DataFrame(np.asarray(y_test).astype('float32'),columns = ['measured tR (min)'])

results_ANN = pd.concat([y_LOO_mea,y_LOO_pred],axis=1)

#outputs_ANN = pd.DataFrame()
#outputs_ANN['pred tR by ANN(min)']=y_LOO_pred
#outputs_ANN['measured tR (min)']=y_LOO_mea
#outputs_ANN['LOO column']=LABEL

print(results_ANN.info())
results_ANN.to_csv('results_ANN_LOO_pH.csv')

Zorbax Extend C18
[ 4.8  1.9 10. ]
2/2 [==============================] - 0s 0s/step
{'test column': 'Zorbax Extend C18', 'test pH': array([ 4.8,  1.9, 10. ]), 'RMSE': 0.4947274642300668}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   measured tR (min)     46 non-null     float32
 1   pred tR by ANN (min)  46 non-null     float32
dtypes: float32(2)
memory usage: 496.0 bytes
None
